In [60]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Add project root to sys.path to import src
project_root = Path.cwd().parent 
sys.path.append(str(project_root))

# Now we can import our functions
from src.recommender import load_artifacts, recommend
from src.preprocessing import preprocess_and_save
from src.vectorizer import train_and_save_vectorizer

Data Preparation

In [63]:
print("⚙️ Preprocessing data...")
preprocess_and_save()  # Creates preprocessed data

print("\n⚙️ Training vectorizer...")
train_and_save_vectorizer()  # Creates TF-IDF model

⚙️ Preprocessing data...
✅ Preprocessed data saved to /Users/paologiovanetti/Desktop/GitHub/Repositories/ml-food-buddy-recommender/data/preprocessed/final_recipes.csv.gzip

⚙️ Training vectorizer...
✅ Vectorizer saved to /Users/paologiovanetti/Desktop/GitHub/Repositories/ml-food-buddy-recommender/models/tfidf_vectorizer.pkl
✅ Recipe vectors saved to /Users/paologiovanetti/Desktop/GitHub/Repositories/ml-food-buddy-recommender/models/recipe_vectors.npy


In [64]:
print("\n🔮 Loading recommendation artifacts...")
df, vectorizer, vectors = load_artifacts()
print(f"Loaded {len(df)} recipes!")


🔮 Loading recommendation artifacts...
Loaded 522512 recipes!


Load the Recommendation System

In [65]:
print("\n🔮 Loading recommendation artifacts...")
df, vectorizer, vectors = load_artifacts()
print(f"Loaded {len(df)} recipes!")


🔮 Loading recommendation artifacts...
Loaded 522512 recipes!


Interactive Demo

In [72]:
def interactive_demo():
    print("\n🍳 ML Food Buddy Recommender")
    print("Type ingredients you like (e.g. 'italian pasta')") 
    
    # Get user input
    query = input("\nWhat would you like to cook? > ").strip()
    if query.lower() in ['quit', 'exit']:
        print("Goodbye!")
        return

    # Get filters
    time_filter = input("Time preference? [fast/medium/long/any] > ").strip().lower() 
    calorie_filter = input("Calorie preference? [low/medium/high/any] > ").strip().lower() 

    # Clean filters
    time_filter = None if time_filter == 'any' else time_filter
    calorie_filter = None if calorie_filter == 'any' else calorie_filter

    print(f"\nProcessing your request: '{query}' "
          f"with time='{time_filter}' and calories='{calorie_filter}'...") 

    # Get recommendations
    results = recommend(
        user_prefs=query,
        dataset=df,
        recipe_vectors_matrix=vectors,
        vectorizer=vectorizer,
        time_pref=time_filter,
        calorie_pref=calorie_filter,
        top_n=3,
        return_columns=["Name", "Image_first", "TotalTime_str", 
                        "recipe_instructions_clean", "Calories", "similarity"]
    )

    # Display results
    if 'message' in results.columns:
        print(f"\n❌ {results['message'].iloc[0]}") 
    else:
        print("\n✨ Recommended Recipes:") 
        for i, row in results.iterrows():
            print(f"\n{i+1}. {row['Name']}") 
            print(f"   ⏱ {row['TotalTime_str']} | 🔥 {int(row['Calories'])} calories")
            print(f"   🖼 {row['Image_first']}")
            print(f"   📜 Instructions: {row['recipe_instructions_clean']}")
            print(f"   🔍 Match score: {row['similarity']:.2f}")

In [76]:
# Run the demo
interactive_demo()


🍳 ML Food Buddy Recommender
Type ingredients you like (e.g. 'italian pasta')

Processing your request: 'pasta with eggs' with time='None' and calories='None'...

✨ Recommended Recipes:

4950. Pasta with Garlic and Eggs
   ⏱ 30 minutes | 🔥 453 calories
   🖼 https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/78/50/picFK0mFe.jpg
   📜 Instructions: Cook pasta. Heat the oil in small frying pan, add garlic. Saute for just a moment. Drain pasta, toss all ingredients together. Salt and pepper to taste.
   🔍 Match score: 0.61

38914. Easy pasta
   ⏱ 9 minutes | 🔥 319 calories
   🖼 nan
   📜 Instructions: Fill a pan about 3/4 of the way with water. Boil the water. Put in pasta (I like to leave whatever wont fit sticking out and push it in when the rest is soft so you get long pasta.) Cook until tender. Drain. put back in pan or put in bow. pour sauce over pasta. stir. eat!
   🔍 Match score: 0.61

393308. Pasta Salad
   ⏱ 45 minutes | 🔥 439 calories
   🖼